### 树回归
- 优点：可以对复杂和非线性的数据建模。
- 缺点：结果不易理解。
- 适用数据类型：数值型和标称型数据
#### ID3
- ID3的做法是每次选取当前最佳的特征来分割数据，并按照该特征的所有可能取值来切分。
- 存在问题：切分过于迅速；不能直接处理连续型特征。
#### 树回归的一般方法：
（1）收集数据：可以使用任意方法。  
（2）准备数据：需要数值型数据，标称型数据将被转成二值型数据。  
（3）分析数据：绘出数据的可视化二维图将有助于对数据做出理解和分析，以字典方式生成树。  
（4）训练算法：大部分时间都花费在叶节点树模型的构建上。  
（5）测试算法：使用测试数据上的R2来分析模型的效果。  
（6）使用算法：使用训练出的树做预测，预测结果还可以用来做很多事情。

In [5]:
import numpy as np

"""
函数说明:加载函数

Parameters:
    fileName - 文件名
Returns:
    dataMat- 数据集
"""
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)     #打开文件
    for line in fr.readlines():
        curLine = line.strip().split('\t')  #读取以tab键为分隔符的文件
        fltLine = list(map(float,curLine))    #将每一行的内容保存成一组浮点数
        dataMat.append(fltLine)
    return dataMat      #返回数据集

In [6]:
"""
函数说明:切分函数

Parameters:
    dataSet - 数据集
    feature - 待切分的特征
    value - 待切分特征的某个值
Returns:
    mat0 - 切分好的子集
    mat1 - 切分好的子集
"""
def binSplitDataSet(dataSet, feature, value):
    #当使用布尔数组直接作为下标对象或者元组下标对象中有布尔数组时，
    # 都相当于用nonzero()将布尔数组转换成一组整数数组，然后使用整数数组进行下标运算。
    mat0 = dataSet[np.nonzero(dataSet[:, feature] > value)[0], :]
    mat1 = dataSet[np.nonzero(dataSet[:, feature] <= value)[0], :]
    return mat0,mat1

In [7]:
"""
函数说明:生成叶节点

Parameters:
    dataSet - 数据集
Returns:
    目标变量特征的均值
"""
def regLeaf(dataSet):
    return np.mean(dataSet[:,-1])

"""
函数说明:误差估计函数

Parameters:
    dataSet - 数据集
Returns:
    目标变量的平方误差
"""
def regErr(dataSet):
    return np.var(dataSet[:,-1])*np.shape(dataSet)[0]
"""
函数说明:找到数据的最佳二元切分方式

Parameters:
    dataSet - 数据集
    leafType - 建立叶节点函数
    errType - 误差计算函数
    ops - 树构建所需其他参数
Returns:
    bestIndex - 最好的特征索引
    bestValue - 最好的特征切分值
"""
def chooseBestSplit(dataSet, leafType=regLeaf, errType = regErr, ops=[1,4]):
    tolS = ops[0]   #容许的误差下降值
    tolN = ops[1]   #切分的最少样本数
    if len(set(dataSet[:,-1].T.tolist()[0]))==1:    #倒数第一列转化成list 不重复，则无需切分
        return None,leafType(dataSet)   # 找不到好的切分特征，调用regLeaf直接生成叶结点
    m,n = np.shape(dataSet)
    S = errType(dataSet)    #计算平方误差
    bestS = np.inf; bestIndex = 0; bestValue = 0
    for featIndex in range(n-1):    #遍历数据的每个属性特征
        for splitVal in set((dataSet[:,featIndex].T.A.tolist())[0]): #遍历每个特征里不同的特征值
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)  #对每个特征进行二元分类
            if (np.shape(mat0)[0]<tolN) or (np.shape(mat1)[0]<tolN): continue   #如果低于规定的切分最少样本数，则跳过本次循环
            newS = errType(mat0)+errType(mat1)
            if newS < bestS:    #更新为误差最小的特征
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    if(S - bestS) < tolS:   #如果切分后误差效果下降不大，则取消切分，直接创建叶结点
        return None, leafType(dataSet)
    mat0,mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    # 判断切分后子集大小，小于最小允许样本数停止切分3
    if (np.shape(mat0)[0] < tolN) or (np.shape(mat1)[0] < tolN):
        return None,leafType(dataSet)
    return bestIndex,bestValue  #返回特征索引和用于切分的特征值

In [8]:
"""
函数说明:树的构建函数

Parameters:
    dataSet - 数据集
    leafType - 建立叶节点函数
    errType - 误差计算函数
    ops - 树构建所需其他参数
Returns:
    retTree- 构建好的树
"""
def createTree(dataSet, leafType=regLeaf, errType=regErr, ops=(1,4)):
    feat,val = chooseBestSplit(dataSet, leafType, errType, ops) #切分数据集
    if feat == None: return val #满足停止条件则返回叶节点值
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    lSet,rSet = binSplitDataSet(dataSet, feat, val)
    retTree['left'] = createTree(lSet, leafType, errType, ops)  #递归调用
    retTree['right'] = createTree(rSet, leafType, errType, ops)
    return retTree

In [10]:
myDat = loadDataSet('data\ex00.txt')
myMat = np.mat(myDat)
retTree = createTree(myMat)
retTree

{'spInd': 0,
 'spVal': 0.48813,
 'left': 1.0180967672413792,
 'right': -0.044650285714285719}

- 通过降低决策树的复杂度来避免过拟合的过程称为剪枝。

In [11]:
"""
函数说明:测试输入变量是否是一棵树

Parameters:
    dataSet - 数据集
Returns:
    是否是一棵树
"""
def isTree(obj):
    return (type(obj).__name__=='dict')

"""
函数说明:返回树的平均值

Parameters:
    tree - 树
Returns:
    树的平均值
"""
def getMean(tree):
    if isTree(tree['right']): tree['right'] = getMean(tree['right'])
    if isTree(tree['left']): tree['left'] = getMean(tree['left'])
    return (tree['right']+tree['left'])/2.0

"""
函数说明:树的后剪枝

Parameters:
    tree - 待剪枝的树
    testData - 剪枝所需测试数据
Returns:
    tree- 剪枝好的树
"""
def prune(tree, testData):
    if np.shape(testData)[0] == 0: return getMean(tree)     #判断测试数据是否为空
    # 假设发生过拟合，采用测试数据对树进行剪枝
    if (isTree(tree['right']) or isTree(tree['left'])):     #左右子树非空
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])    #对测试数据生成树
    if isTree(tree['left']): tree['left'] = prune(tree['left'],lSet)    #判断左分支是否为树，若是，则调用剪枝函数进行剪枝
    if isTree(tree['right']): tree['right'] = prune(tree['right'], rSet)#判断右分支是否为树，若是，则调用剪枝函数进行剪枝
    # 剪枝后判断是否还是有子树
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
        # 判断是否merge
        errorNoMerge = np.sum(np.power(lSet[:,-1]-tree['left'],2))+np.sum(np.power(rSet[:,-1]-tree['right'],2))
        treeMean = (tree['left']+tree['right'])/2.0
        errorMerge = np.sum(np.power(testData[:,-1]-treeMean,2))
        # 如果合并后误差变小
        if errorMerge < errorNoMerge:
            print("merging")
            return treeMean
        else: return tree
    else: return tree

- 模型树的可解释性是它优于回归树的特点之一。另外，模型树也具有更高的预测准确度。

In [12]:
"""
函数说明:模型树

Parameters:
    dataSet - 数据集
Returns:
    ws - 回归系数
    X - 自变量X
    Y - 目标变量Y
"""
def linearSolve(dataSet):
    m,n = np.shape(dataSet)
    X = np.mat(np.ones((m,n))); Y = np.mat(np.ones((m,1)))
    X[:,1:n] = dataSet[:,0:n-1]; Y = dataSet[:,-1]  #将数据集格式化为X,Y
    xTx = X.T*X
    if np.linalg.det(xTx) == 0.0:   #X,Y应用简单线性回归求得回归系数ws
        raise NameError("This matrix is singular, cannon do inverse,\n"
                        "try increasing the second value of ops")
    ws = xTx.I*(X.T*Y)
    return ws,X,Y

"""
函数说明:生成叶节点的模型

Parameters:
    dataSet - 数据集
Returns:
    ws - 回归系数
"""
def modelLeaf(dataSet):
    ws,X,Y = linearSolve(dataSet)
    return ws

"""
函数说明:在给定的数据集上计算误差

Parameters:
    dataSet - 数据集
Returns:
    误差
"""
def modelErr(dataSet):
    ws,X,Y = linearSolve(dataSet)
    yHat = X*ws
    return np.sum(np.power(Y-yHat,2))

#### 示例：树回归与标准回归的比较

In [13]:
"""
函数说明:回归树叶节点预测

Parameters:
    model - 数据集
    inDat - 原数据矩阵
Returns:
    预测值
"""
def regTreeEval(model, inDat):
    return float(model)

"""
函数说明:模型树叶节点预测

Parameters:
    model - 数据集
    inDat - 原数据矩阵
Returns:
    预测值
"""
def modelTreeEval(model, inDat):
    n = np.shape(inDat)[1]
    X = np.mat(np.ones((1,n+1)))    #在原数据矩阵上增加第0列
    X[:,1:n+1] = inDat
    return float(X*model)   #计算并返回预测值

"""
函数说明:预测

Parameters:
    tree - 训练好的树
    inDat - 待预测数据，单个数据点或行向量
    modelEval - 树的类型
Returns:
    预测值
"""
def treeForeCast(tree, inData, modelEval=regTreeEval):
    if not isTree(tree): return modelEval(tree,inData)
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']):
            return treeForeCast(tree['left'],inData,modelEval)
        else:
            return modelEval(tree['left'],inData)
    else:
        if isTree(tree['right']):
            return treeForeCast(tree['right'],inData,modelEval)
        else:
            return modelEval(tree['right'],inData)

"""
函数说明:以向量形式返回一组预测值

Parameters:
    tree - 训练好的树
    testData - 待预测数据矩阵
    modelEval - 树的类型
Returns:
    预测值
"""
def creatForeCast(tree, testData, modelEval = regTreeEval):
    m = len(testData)   #测试数据的行数
    yHat = np.mat(np.zeros((m,1)))
    for i in range(m):
        yHat[i,0] = treeForeCast(tree, np.mat(testData[i]), modelEval)
    return yHat

- 创建一棵回归树

In [14]:
trainMat = np.mat(loadDataSet("data\\bikeSpeedVsIq_train.txt"))
testMat = np.mat(loadDataSet("data\\bikeSpeedVsIq_test.txt"))
myTree = createTree(trainMat,ops=(1,20))
yHat = creatForeCast(myTree,testMat[:,0])
np.corrcoef(yHat,testMat[:,1],rowvar=0)[0,1]

0.96408523182221495

- 创建一棵模型树

In [18]:
myTree = createTree(trainMat,modelLeaf,modelErr, ops=(1, 20))
yHat = creatForeCast(myTree, testMat[:, 0],modelTreeEval)
np.corrcoef(yHat, testMat[:, 1], rowvar=0)[0, 1]

0.97604121913806285

- 由上可得，模型树的结果比回归树的结果好。

- 标准线性回归效果如何?

In [20]:
ws, X, Y = linearSolve(trainMat)
for i in range(np.shape(testMat)[0]):
    yHat[i]=testMat[i,0]*ws[1,0]+ws[0,0]
np.corrcoef(yHat,testMat[:,1],rowvar=0)[0, 1]

0.94346842356747662

- 树回归方法在预测复杂数据时会比简单的线性模型更有效。